In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from numpy.random import randint
import pandas as pd
my_NIA = 100419401
np.random.seed(my_NIA)
#3.Selecting the first four points,300 attributes)
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X_train = train.iloc[:,0:300].values
y_train = train.iloc[:,1200:1201].values
X_test = test.iloc[:,0:300].values
y_test = test.iloc[:,1200:1201].values

In [2]:
#4 .10% of columns300*10%=30
selectedcolumns=np.random.choice(300, 30, replace=False)
selectedcolumns

array([203,  63,  46, 199,  40,  34,  65, 180, 120,  77, 174, 232,  49,
        70,  88,  47, 136, 186, 118, 258, 107, 129, 248, 282, 108, 111,
        55,  31,  32,  89])

In [3]:
for column in selectedcolumns:
    rplace1=np.random.choice(4380, 438, replace=False)
    for place in  rplace1:
        X_train[place,column]=np.nan
for column in selectedcolumns:
    rplace2=np.random.choice(733, 73, replace=False)
    for place in  rplace2:
        X_test[place,column]=np.nan

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
#4.validation using PredefinedSplit in part1
from sklearn.model_selection import PredefinedSplit
validation_indices = np.zeros(X_train.shape[0])
validation_indices[:round(10/12*X_train.shape[0])] = -1
tr_val_partition = PredefinedSplit(validation_indices)

imputer=SimpleImputer(strategy='median')
remove=VarianceThreshold(threshold=1)
scaler = preprocessing.MinMaxScaler()
pca=PCA()
knn=KNeighborsRegressor()
estimators = [('impute', imputer), ('remove', remove),('scaler',scaler),("pca",pca), ('knn', knn)]
pcapipeline=Pipeline(estimators)
param_grid={'n_components':range(1,40,1)}

In [5]:
from sklearn.model_selection import GridSearchCV
pca_grid=GridSearchCV(pcapipeline,param_grid,scoring='neg_mean_squared_error',cv=tr_val_partition,n_jobs=-1, verbose=1)

In [6]:
pca_grid=pca_grid.fit(X_train,y_train)
y_test_pca_pred=pca_grid.predict(X_test)
pca_grid.best_params_
pca_grid.best_score_

Fitting 1 folds for each of 39 candidates, totalling 39 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter n_components for estimator Pipeline(memory=None,
         steps=[('impute',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('remove', VarianceThreshold(threshold=1)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('knn',
                 KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                     metric='minkowski', metric_params=None,
                                     n_jobs=None, n_neighbors=5, p=2,
                                     weights='uniform'))],
         verbose=False). Check the list of available parameters

In [7]:
#pipeline2 SelectkBest
from sklearn.feature_selection import SelectKBest,f_regression
selector=SelectKBest(f_regression)
estimators2 = [('impute', imputer), ('remove', remove),('scaler',scaler),("select",selector), ('knn', knn)]
selectpipeline=Pipeline(estimators2)
param_grid2={'select_k':range(1,40,1)}
from sklearn.model_selection import GridSearchCV
select_grid=GridSearchCV(selectpipeline,param_grid2,scoring='neg_mean_squared_error',cv=tr_val_partition,n_jobs=-1, verbose=1)
select_grid=select_grid.fit(X_train,y_train)
y_test_select_pred=select_grid.predict(X_test)
select_grid.best_params_
select_grid.best_score_

Fitting 1 folds for each of 39 candidates, totalling 39 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter select_k for estimator Pipeline(memory=None,
         steps=[('impute',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('remove', VarianceThreshold(threshold=1)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('select',
                 SelectKBest(k=10,
                             score_func=<function f_regression at 0x000001586387F828>)),
                ('knn',
                 KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                     metric='minkowski', metric_params=None,
                                     n_jobs=None, n_neighbors=5, p=2,
                                     weights='uniform'))],
         verbose=False). Check the list of available parameters with `estimator.get_params().keys()`.